In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,confusion_matrix

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

2023-04-25 06:52:31.400537: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 06:52:33.131878: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 06:52:33.144973: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 06:52:49.747591: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Set up the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_labels = pd.read_csv('train_img/_annotations.csv')
# Convert the column to strings
train_labels['class'] = train_labels['class'].astype(str)
train_generator = train_datagen.flow_from_dataframe(
    train_labels,
    directory='train_img',
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 771 validated image filenames belonging to 3 classes.


In [5]:
# Load the pre-trained DenseNet-169 model and freeze the base layers
base_model = tf.keras.applications.DenseNet169(
    include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add new fully connected layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Compile the model with binary cross-entropy loss and Adam optimizer
model = Model(inputs=base_model.input, outputs=predictions)

#freeze Layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20)

51877672/51877672 [==============================] - 13s 0us/step


2023-04-25 06:55:52.913622: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/20
24/24 [==============================] - 35s 1s/step - loss: 0.6955 - accuracy: 0.6207
Epoch 2/20
24/24 [==============================] - 27s 1s/step - loss: 0.6457 - accuracy: 0.6613
Epoch 3/20
24/24 [==============================] - 27s 1s/step - loss: 0.6417 - accuracy: 0.6658
Epoch 4/20
24/24 [==============================] - 27s 1s/step - loss: 0.6404 - accuracy: 0.6658
Epoch 5/20
24/24 [==============================] - 28s 1s/step - loss: 0.6409 - accuracy: 0.6658
Epoch 6/20
24/24 [==============================] - 28s 1s/step - loss: 0.6394 - accuracy: 0.6667
Epoch 7/20
24/24 [==============================] - 28s 1s/step - loss: 0.6391 - accuracy: 0.6667
Epoch 8/20
24/24 [==============================] - 28s 1s/step - loss: 0.6388 - accuracy: 0.6667
Epoch 9/20
24/24 [==============================] - 28s 1s/step - loss: 0.6385 - accuracy: 0.6667
Epoch 10/20
24/24 [==============================] - 28s 1s/step - loss: 0.6387 - accuracy: 0.6667
Epoch 11/20
24/24 [

In [6]:
# Load the test CSV file with filenames
t_test_df = pd.read_csv("test_img/_annotations.csv")

In [7]:
# Create a generator for the test images
t_test_datagen = ImageDataGenerator(rescale=1./255)
t_test_generator = t_test_datagen.flow_from_dataframe(
    t_test_df,
    directory="test_img",
    x_col='filename',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False)

# Use the trained model to make predictions on the test data
preds = model.predict_generator(t_test_generator, steps=None)
# Remove any missing filenames from the test CSV file
missing_filenames = set(t_test_generator.filenames) - set(t_test_df['filename'])
if missing_filenames:
    print(f'Removing {len(missing_filenames)} missing filenames from the test CSV file.')
    t_test_df = t_test_df[~t_test_df['filename'].isin(missing_filenames)]

Found 771 validated image filenames.


/home/u190377/tmp/ipykernel_3193743/1759325062.py:14: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  preds = model.predict_generator(t_test_generator, steps=None)
2023-04-25 07:07:45.286543: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [8]:
li = []
for i in preds.ravel():
    if i>0.5:
        li.append(1)
    else:
        li.append(0)
     
# Create a DataFrame with the filenames and predictions
df = pd.DataFrame({'filename': t_test_generator.filenames[:len(preds)], 'class': li})

test_inst_df=pd.merge(df, t_test_df, on='filename', how='inner')

In [9]:
accuracy_score(df["class"],t_test_df['class'])

0.5136186770428015

In [10]:
confusion_matrix(df["class"],t_test_df['class']) 

array([[396, 214, 161],
       [  0,   0,   0],
       [  0,   0,   0]])

In [11]:
# Save the model
model.save('models2/densenet.h5')

In [12]:
# Saving the model for Future Inferences

model_json = model.to_json()
with open("densenetmodel.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("densenetmodel.h5")

In [13]:
#Saving Trained Models With h5py
from keras.models import model_from_json 

# opening and store file in a variable

json_file = open('densenetmodel.json','r')
loaded_model_json = json_file.read()
json_file.close()

# use Keras model_from_json to make a loaded model

loaded_model = model_from_json(loaded_model_json)

# load weights into new model

loaded_model.load_weights("densenetmodel.h5")
print("Loaded Model from disk")

# compile and evaluate loaded model

loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


Loaded Model from disk
